## **Machine learning Project**
### By **Thomas Marchal** #20220714, **Louis Hislaire** #  and **Jules Rougier** # 


Step 1:
We choose the Dataset number D : House_data


Step 2: Scenario / About Dataset

Step 3: Data Loading


Step 4: Data Wrangling or Data Pre-processing

Step 5: Exploratory Data Analysis

Step 6: Model Developement (Apply ML algorithms)
BONUS if we choose to use 2ML algorithms (see pdf)

Step 7: Model Evaluation, Implement and compare different metrics to evaluate the quality of the model.

Step 8: Model Refinement